In [ ]:
import os
# ============================================================
# 1. CONFIGURATION (CRUCIAL : AVANT LES IMPORTS)
# ============================================================
# Désactive le vérificateur de type strict qui causait les plantages
os.environ["JAXTYPING_DISABLE"] = "1" 

import torch
import skshapes as sks
import pyvista as pv
import numpy as np

# ============================================================
# 2. GÉNÉRATION DES DONNÉES (MODE 3D APLATI)
# ============================================================
print("--- Génération des formes ---")

# On utilise sks.Sphere() qui génère un maillage 3D valide (points + triangles)
# Cela évite les erreurs de dimension avec la VarifoldLoss
base_sphere = sks.Sphere()

# --- FORME 1 : SOURCE (Pré-op : Foie + Tumeur) ---
# On crée une nouvelle sphère
liver1 = sks.Sphere() 
# On l'aplatit (Z=0) et on l'étire (X=10, Y=8)
liver1.points = liver1.points * torch.tensor([10.0, 8.0, 0.0]) 
liver1.points = liver1.points + torch.tensor([0.0, 0.0, 0.0]) # Centré en 0,0

# --- FORME 2 : CIBLE (Post-op : Foie Rétracté) ---
liver2 = sks.Sphere()
# Plus petit (X=8, Y=6) et décalé
liver2.points = liver2.points * torch.tensor([8.0, 6.0, 0.0])
liver2.points = liver2.points + torch.tensor([0.5, 0.0, 0.0]) 

# Assignation explicite
source = liver1
target = liver2

# ============================================================
# 3. CONFIGURATION DU RECALAGE
# ============================================================
print("--- Configuration du modèle ---")

# A. Points de contrôle (Grille)
# On génère la grille autour de la source
source.control_points = source.bounding_grid(N=10, offset=0.5)

# B. Correction de la grille (Sécurité 2D -> 3D)
# Si la grille générée est plate (N, 2), on force (N, 3) pour éviter le crash
if source.control_points.points.shape[1] == 2:
    print("   -> Conversion de la grille de contrôle en 3D...")
    cp = source.control_points.points
    source.control_points.points = torch.cat([cp, torch.zeros(cp.shape[0], 1)], dim=1)

# C. Modèle de Déformation (LDDMM)
model = sks.ExtrinsicDeformation(
    n_steps=10,             # Fluidité de la déformation
    kernel="gaussian",      # Type de noyau
    scale=5.0,              # Rigidité (plus grand = déformation plus globale)
    control_points=True
)

# D. Fonction de Coût (Varifold)
# Gestion robuste des versions de la librairie
try:
    loss = sks.VarifoldLoss(radial_bandwidth=2.0)
except (AttributeError, TypeError):
    loss = sks.Loss(loss_type="varifold", scales=[2.0])

# E. Objet Registration
registration = sks.Registration(
    model=model,
    loss=loss,
    optimizer=sks.LBFGS(),
    n_iter=15,              # Nombre d'itérations
    verbose=True,           # Afficher la progression
    regularization_weight=0.1 # Pénalité de déformation (0.1 = équilibré)
)

# ============================================================
# 4. EXÉCUTION
# ============================================================
print("--- Démarrage du recalage ---")
morphed = registration.fit_transform(source=source, target=target)
print("--- Terminé ! ---")

# ============================================================
# 5. VISUALISATION INTERACTIVE (PYVISTA)
# ============================================================
print("--- Ouverture de la fenêtre 3D ---")

# Conversion en objets PyVista
pv_target = target.to_pyvista()   # Cible (Bleu)
pv_source = source.to_pyvista()   # Source (Gris/Fil de fer)
pv_morphed = morphed.to_pyvista() # Résultat (Rouge)

# Création de la scène (2 vues côte à côte)
plotter = pv.Plotter(shape=(1, 2))

# Vue de Gauche : Avant
plotter.subplot(0, 0)
plotter.add_text("AVANT Recalage", font_size=10)
plotter.add_mesh(pv_target, color="blue", opacity=0.3, label="Cible (Post-op)")
plotter.add_mesh(pv_source, color="silver", style="wireframe", line_width=2, label="Source (Pré-op)")
plotter.add_legend()

# Vue de Droite : Après
plotter.subplot(0, 1)
plotter.add_text("APRÈS Recalage", font_size=10)
plotter.add_mesh(pv_target, color="blue", opacity=0.3, label="Cible")
plotter.add_mesh(pv_morphed, color="red", opacity=0.8, show_edges=True, label="Recalé")
plotter.add_legend()

# Lier les caméras pour zoomer sur les deux en même temps
plotter.link_views()

plotter.show()

[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.
